In [52]:
with open('day22_input.txt') as file:
    puzzle_input = file.read()

def parse_instructions(path):
    instructions = []
    length_buffer = ''
    for char in path:
        if '0' <= char <= '9':
            length_buffer += char
        elif char == 'L':
            if length_buffer:
                instructions.append({'op': 'move', 'dist': int(length_buffer)})
                length_buffer = ''
            instructions.append({'op': 'left'})
        elif char == 'R':
            if length_buffer:
                instructions.append({'op': 'move', 'dist': int(length_buffer)})
                length_buffer = ''
            instructions.append({'op': 'right'})
            
    if length_buffer:
        instructions.append({'op': 'move', 'dist': int(length_buffer)})

    return instructions

def parse_puzzle_input(puzzle_input):
    lines = puzzle_input.splitlines()
    instructions = parse_instructions(lines[-1])
    max_width = max(len(line) for line in lines[:-2])
    grid = []
    for line in lines[:-2]:
        grid.append(list(line) + [' '] * (max_width - len(line)))
    
    return grid, instructions
    
def calc_start_pos(grid):
    for y, line in enumerate(grid):
        for x, line in enumerate(grid):
            if grid[y][x] != ' ':
                return (x, y)    

def process_instruction(grid, instruction, x, y, direction):
    offsets = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    width = len(grid[0])
    height = len(grid)

    if instruction['op'] == 'left':
        direction = (direction - 1 + len(offsets)) % len(offsets)
    elif instruction['op'] == 'right':
        direction = (direction + 1) % len(offsets)
    elif instruction['op'] == 'move':
        for _ in range(instruction['dist']):
            next_x = x + offsets[direction][0]
            next_y = y + offsets[direction][1]
            
            if next_x < 0:
                next_x = width - 1
            elif next_x >= width:
                next_x = 0
            
            if next_y < 0:
                next_y = height - 1
            elif next_y >= height:
                next_y = 0
                
            while grid[next_y][next_x] == ' ':
                next_x += offsets[direction][0]
                next_y += offsets[direction][1]

                if next_x < 0:
                    next_x = width - 1
                elif next_x >= width:
                    next_x = 0

                if next_y < 0:
                    next_y = height - 1
                elif next_y >= height:
                    next_y = 0
            
            if grid[next_y][next_x] == '#':
                break
            
            x = next_x
            y = next_y
    
    return x, y, direction

def calc_password(grid, x, y, direction):
    row = y + 1
    column = x + 1
    return 1000 * row + 4 * column + direction
    
grid, instructions = parse_puzzle_input(puzzle_input)
direction = 0
x, y = calc_start_pos(grid)
for instruction in instructions:
    x, y, direction = process_instruction(grid, instruction, x, y, direction)
    
print(calc_password(grid, x, y, direction))

57350


In [202]:
import numpy as np
from scipy.spatial.transform import Rotation

puzzle_input = '''        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5
'''

with open('day22_input.txt') as file:
    puzzle_input = file.read()

grid, instructions = parse_puzzle_input(puzzle_input)
grid_width = len(grid[0])
grid_height = len(grid)
grid = np.array(grid).T
block_size = 50
cube_size = (block_size + 4)
cube = np.chararray((cube_size, cube_size, cube_size))
cube[:] = '_'
visited_grid_poses = set()
grid_offsets = np.array([
    [1, 0],
    [0, 1],
    [-1, 0],
    [0, -1]
])
cube_pos_to_grid_pos = {}
cube_pos_to_origin_cube_offsets = {}

def will_jump_edge(cube_pos):
    return bool(set(cube_pos) & {1, cube_size - 2})

def find_face_normal(cube_pos):
    for dimension, coord in enumerate(cube_pos):
        if coord == 0:
            return np.eye(*cube_pos.shape, dtype=int)[dimension]
        elif coord == cube_size - 1:
            return -np.eye(*cube_pos.shape, dtype=int)[dimension]

def jump_edge(cube_pos):
    jump_map = {0: 2, 1: 0, cube_size - 2: cube_size - 1, cube_size - 1: cube_size - 3}
    jumped_cube_pos = np.zeros(cube_pos.shape, dtype=int)
 
    for dimension in range(*cube_pos.shape):
        jumped_cube_pos[dimension] = jump_map.get(cube_pos[dimension], cube_pos[dimension])

    return jumped_cube_pos
    
def try_jump_edge(cube_pos):
    jumped_cube_pos = jump_edge(cube_pos)
    
    old_face_normal = find_face_normal(cube_pos)
    new_face_normal = find_face_normal(jumped_cube_pos)
    rotation_axis = np.cross(new_face_normal, old_face_normal)
    
    return jumped_cube_pos, rotation_axis

def apply_rotation(cube_offsets, cube_pos):
    if not will_jump_edge(cube_pos):
        return cube_offsets, cube_pos
    
    rotated_cube_pos, rotation_axis = try_jump_edge(cube_pos)
    rotation_matrix = Rotation.from_rotvec(90 * rotation_axis, degrees=True).as_matrix().astype(int)
    rotated_cube_offsets = cube_offsets @ rotation_matrix
    return rotated_cube_offsets, rotated_cube_pos

def fill_cube(init_cube_offsets, init_cube_pos, init_grid_pos):
    queue = [(init_cube_offsets, init_cube_pos, init_grid_pos)]
    
    while queue:
        cube_offsets, cube_pos, grid_pos = queue.pop(0)
    
        if tuple(grid_pos) in visited_grid_poses:
            continue

        visited_grid_poses.add(tuple(grid_pos))

        grid_x, grid_y = grid_pos
        if grid_x < 0 or grid_x >= grid_width or grid_y < 0 or grid_y >= grid_height:
            continue

        if grid[tuple(grid_pos)] == ' ':
            continue

        cube_offsets, cube_pos = apply_rotation(cube_offsets, cube_pos)

        assert cube[tuple(cube_pos)] == b'_'
        cube[tuple(cube_pos)] = grid[tuple(grid_pos)]
        cube_pos_to_grid_pos[tuple(cube_pos)] = grid_pos
        cube_pos_to_origin_cube_offsets[tuple(cube_pos)] = cube_offsets

        for grid_offset, cube_offset in zip(grid_offsets, cube_offsets):
            queue.append((cube_offsets, cube_pos + cube_offset, grid_pos + grid_offset))
            
init_grid_pos = np.array(calc_start_pos(grid.T))
init_cube_pos = np.array([2, 2, 0])
init_cube_offsets = np.c_[grid_offsets, np.zeros(grid_offsets.shape[0], dtype=int)]

fill_cube(init_cube_offsets, init_cube_pos, init_grid_pos)

def print_cube():
    def format_face(face):
        return '\n'.join(''.join(cell.decode("utf-8") for cell in row) for row in face)
    
    print('up')
    print(format_face(cube[2:cube_size-2, 2:cube_size-2, 0].T))
    print('down')
    print(format_face(np.rot90(cube[2:cube_size-2, 2:cube_size-2, -1])))
    print('left')
    print(format_face(cube[0, 2:cube_size-2, 2:cube_size-2].T))
    print('right')
    print(format_face(np.rot90(np.rot90(cube[-1, 2:cube_size-2, 2:cube_size-2]))))
    print('back')
    print(format_face(np.rot90(cube[2:cube_size-2, 0, 2:cube_size-2], k=-1)))
    print('front')
    print(format_face(cube[2:cube_size-2, -1, 2:cube_size-2].T))

def run_instructions():
    cube_pos = init_cube_pos
    cube_direction = 0
    cube_offsets = init_cube_offsets.copy()
    
    for instruction in instructions:
        if instruction['op'] == 'left':
            cube_direction = (cube_direction + len(grid_offsets) - 1) % len(grid_offsets)
        elif instruction['op'] == 'right':
            cube_direction = (cube_direction + 1) % len(grid_offsets)
        elif instruction['op'] == 'move':
            for _ in range(instruction['dist']):
                next_cube_offsets, next_cube_pos = apply_rotation(cube_offsets,
                      cube_pos.copy() + cube_offsets[cube_direction])
                if cube[tuple(next_cube_pos)] == b'#':
                    break
                else:
                    cube_offsets = next_cube_offsets
                    cube_pos = next_cube_pos
                    #x, y = cube_pos_to_grid_pos[tuple(cube_pos)]
                    #grid[x, y] = b'v'
    return cube_pos, cube_offsets[cube_direction]

def calc_score(grid_pos, direction):
    x, y = grid_pos
    row = x + 1
    column = y + 1
    return 1000 * column + 4 * row + direction

def cube_to_grid_direction(cube_pos, cube_direction_vector):
    grid_directions = np.where((cube_pos_to_origin_cube_offsets[tuple(cube_pos)] == cube_direction_vector).all(axis=1))
    return cube_pos_to_grid_pos[tuple(cube_pos)], grid_directions[0][0]

cube_pos, cube_direction_vector = run_instructions()
grid_pos, grid_direction = cube_to_grid_direction(cube_pos, cube_direction_vector)
print(calc_score(grid_pos, grid_direction))


104385
